In [ ]:
%env OPENAI_API_KEY=keyhere

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from kagent.tools.docs import Config, QueryTool

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

docs_config = Config(
    docs_path="",  # empty string means the database will be downloaded
)

cncf_docs_agent = AssistantAgent(
    "kubernetes_docs_agent",
    description="This agent allows you to get data from the CNCF docs databases.",
    tools=[
        QueryTool(config=docs_config),
    ],
    model_client=model_client,
    system_message="""
    You are a support agent.

    You have access to several tools:
    - 'searchDocumentation' to search in the documentation.

    Execute all the following steps:
    1. Product identification
    - Check if you know what product the question is about (Only supported products are: 'kubernetes', 'istio', 'argo', 'helm', 'prometheus')
    - If it's not the case, ask what the product is

    2. Information Gathering
    - Search in the documentation for information related to the question the user has submitted

    3. Answer the question
    - Use all the information you gathered to provide a valuable answer to the user
    - Provide links to the documentation whenever possible`,
    """,
)

user_proxy = UserProxyAgent("user_proxy", input_func=input)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = RoundRobinGroupChat(
    [
        cncf_docs_agent,
    ],
    termination_condition=termination,
)

# task = "How do I do a canary upgrade of a pod in kubernetes using argo rollouts?"
# task = "How to I install a helm chart in it doesn't exist or upgrade it if it does?"
task = "Write a Prometheus query to check if Istio requests are using mTLS?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Write a Prometheus query to check if Istio requests are using mTLS?
---------- kubernetes_docs_agent ----------
[FunctionCall(id='call_OTifhhMG9Em7Mjgnlb0YfFin', arguments='{"query": "check Istio requests using mTLS Prometheus query", "product_name": "istio"}', name='QueryTool'), FunctionCall(id='call_KQjDfWqkCJuXMzNAKGncNsMT', arguments='{"query": "monitor mTLS status with Prometheus", "product_name": "prometheus"}', name='QueryTool')]
---------- kubernetes_docs_agent ----------
[FunctionExecutionResult(content='[{\'distance\': 0.9378427863121033, \'content\': \'\\nThe control plane, gateway, and Envoy sidecar metrics will all be scraped over cleartext. However, the application metrics will follow whatever [Istio authentication policy](/latest/docs/tasks/security/authentication/authn-policy/) has been configured for the workload.\\n\\n*   If you use `STRICT` mode, then Prometheus will need to be configured to scrape using Istio certificates as described belo

In [ ]:
print(team.dump_component().model_dump_json(indent=2))